In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import model_from_json

ModuleNotFoundError: No module named 'tensorflow.python.platform'

In [ ]:
loaded_model.load_weights('model.h5')

In [ ]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
loaded_model.summary()

In [ ]:
def sentiment_analysis(con):
  test_cons = [con]
  test_cons = [remove_stop_words(con) for con in test_cons]
  test_seq = tokenizer.texts_to_sequences(test_cons)
  padded_test_seq = pad_sequences(test_seq, maxlen=max_length, truncating='post', padding='post') 
  result_one_hot_predicted = model.predict(padded_test_seq)
  rating_predicted = np.argmax(result_one_hot_predicted)
  if (rating_predicted >= 2):
    return True
  return False
